In [2]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import kerastuner as kt
import keras_tuner as kt


# Directories where your input images are located (train, test, and validation)
train_dir = r"F:\Thesis\0.WorkHere\Train"
test_dir = r"F:\Thesis\0.WorkHere\Test"
validation_dir = r"F:\Thesis\0.WorkHere\Validation"

# Directory to save models and history
save_dir = r"F:\Thesis\0.WorkHere\SavedModels\Tuned Models"

# Set hyperparameters
batch_size = 5
num_classes = 169
epochs = 20



C:\Users\sayed\AppData\Local\Temp\ipykernel_752\3944463045.py:11: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [2]:
# Create data generators for image preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create data generators for training, testing, and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 5509 images belonging to 169 classes.
Found 568 images belonging to 169 classes.
Found 986 images belonging to 169 classes.


In [6]:

# Function to build the model with hyperparameters
def build_model(hp):
    base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model_vgg19.trainable = False  # Freeze the base model

    # Define hyperparameters
    hp_units = hp.Int('units', min_value=512, max_value=2048, step=512)
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model = models.Sequential([
        base_model_vgg19,
        layers.Flatten(),
        layers.Dense(units=hp_units, activation='relu'),
        layers.Dropout(rate=hp_dropout),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp_learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [7]:

# Initialize the tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    hyperband_iterations=2,
    directory='hyperband',
    project_name='keras_tuner_demo'
)

# Perform hyperparameter search
tuner.search(train_generator,
             validation_data=validation_generator,
             epochs=10,
             callbacks=[ModelCheckpoint(
                 filepath=save_dir + '/vgg19_model_{epoch}.h5',
                 monitor='val_loss',
                 save_best_only=True,
                 verbose=1
             )])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal dropout rate is {best_hps.get('dropout')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=[ModelCheckpoint(
        filepath=save_dir + '/vgg19_model_tuned.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )]
)



Trial 60 Complete [00h 08m 20s]
val_accuracy: 0.01217038556933403

Best val_accuracy So Far: 0.7809330821037292
Total elapsed time: 03h 25m 09s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 2048 and the optimal dropout rate is 0.0.
The optimal learning rate for the optimizer is 0.0001.

Epoch 1/20
1105/1106 [============================>.] - ETA: 0s - loss: 4.8463 - accuracy: 0.0481
Epoch 1: val_loss improved from inf to 4.25244, saving model to F:\Thesis\0.WorkHere\SavedModels\Tuned Models\vgg19_model_tuned.h5
1106/1106 [==============================] - 52s 46ms/step - loss: 4.8462 - accuracy: 0.0481 - val_loss: 4.2524 - val_accuracy: 0.1197
Epoch 2/20
1106/1106 [==============================] - ETA: 0s - loss: 3.8541 - accuracy: 0.1722
Epoch 2: val_loss improved from 4.25244 to 3.25136, saving model to F:\Thesis\0.WorkHere\SavedModels\Tuned Models\vgg19_model_tuned.h5
1106/1106 [==============================] - 53s 48ms

In [5]:
# # Save training history to a JSON file
# with open(save_dir + '/vgg19_history.json', 'w') as f:
#     json.dump(history.history, f)

# Load the model and history for evaluation without retraining
model_vgg19_loaded = load_model(save_dir + '/vgg19_model_tuned.h5')

# # Load the history
# with open(save_dir + '/vgg19_history.json', 'r') as f:
#     history_vgg19_loaded = json.load(f)



In [11]:
# Evaluate the model on the test data
test_loss, test_acc = model_vgg19_loaded.evaluate(test_generator)
print(f'Test accuracy vgg19: {test_acc * 100:.2f}%')

# Evaluate the model on the validation data
validation_loss, validation_acc = model_vgg19_loaded.evaluate(validation_generator)
print(f'Validation accuracy vgg19: {validation_acc * 100:.2f}%')



114/114 [==============================] - 9s 73ms/step - loss: 0.4668 - accuracy: 0.8785
Test accuracy vgg19: 87.85%
198/198 [==============================] - 4s 22ms/step - loss: 0.4551 - accuracy: 0.8732
Validation accuracy vgg19: 87.32%


In [12]:
# Generate predictions
predictions = model_vgg19_loaded.predict(test_generator)

# Convert predicted probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get true labels from the test set generator
true_labels = test_generator.classes

# Get class labels (optional, but useful for the classification report)
class_labels = list(test_generator.class_indices.keys())

# Ensure the classification report and confusion matrix consider all classes
labels = list(test_generator.class_indices.values())

# Print classification report
print(classification_report(true_labels, predicted_labels, labels=labels, target_names=class_labels, zero_division=1))

# Print confusion matrix
print(confusion_matrix(true_labels, predicted_labels, labels=labels))


114/114 [==============================] - 3s 22ms/step
              precision    recall  f1-score   support

       cow_1       1.00      0.00      0.00         2
      cow_10       0.00      0.00      0.00         4
     cow_101       1.00      0.00      0.00         2
     cow_110       0.00      0.00      0.00         2
     cow_112       0.00      0.00      0.00         4
     cow_113       0.00      0.00      0.00         4
     cow_118       0.00      0.00      0.00         2
     cow_119       0.00      0.00      0.00         4
     cow_120       0.00      0.00      0.00         4
     cow_121       0.00      0.00      0.00         4
     cow_122       0.00      0.00      0.00         4
     cow_123       0.08      0.17      0.11         6
     cow_124       0.25      0.25      0.25         4
     cow_126       0.00      0.00      0.00         4
     cow_127       0.00      0.00      0.00         6
     cow_129       0.00      0.00      0.00         2
      cow_13       0.00  

In [9]:
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Function to load and preprocess the image
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values to [0, 1]
    return img_array

# Load the pre-trained model
model_path = os.path.join(save_dir, 'vgg19_model_tuned.h5')
model_InceptionV3_loaded = tf.keras.models.load_model(model_path)

# Path to the image to predict
img_path = r"F:\Thesis\1.bigDataSet\Final Dataset\cow_20\cow_20_1.jpg"

# Preprocess the image
preprocessed_img = load_and_preprocess_image(img_path)

# Make the prediction
prediction = model_vgg19_loaded.predict(preprocessed_img)

# Get the class indices from the training generator
class_indices = train_generator.class_indices
# Reverse the class indices to get a mapping from indices to class names
indices_to_class = {v: k for k, v in class_indices.items()}

# Get the predicted class index
predicted_class_index = np.argmax(prediction[0])
# Get the predicted class name
predicted_class_name = indices_to_class[predicted_class_index]

# Print the predicted class name
print(f"Predicted class: {predicted_class_name}")


1/1 [==============================] - 0s 16ms/step
Predicted class: cow_310
